In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import re

In [2]:
# формируем исходный фрейм из выписки, путь подгружается в ручном режиме
df = pd.read_html('/Users/artem/Downloads/ЛС277394263745_номер79160331018_с_01092022_по_30092022.html')
data = pd.DataFrame(df[0]) # выделяем лист с данными и формируем колонки
cols = ['temp', 'date', 'time',  'GMT', 'number', 'session_num', 'connection_type', 'unknown', 'costs', 'long', 'unnamed_1', 'index']
data.columns = cols
data = data.drop(['temp', 'GMT'], axis=1)
data = data.reset_index(drop=True)

In [3]:
for i in range(len(data['number'])):
    if str(data.loc[i, 'number']).__contains__('<--') == True: # убираем символ-обозначение ВХОДЯЩЕГО вызова - в статистике используем только исходящие
        data = data.drop(index=i, axis=0)
data = data.reset_index(drop=True)

In [4]:
pattern = re.compile('[a-zA-Z#*]') # убираем буквы и ненужные символы типа *
for i in range(len(data['number'])):
    if pattern.match(str(data.loc[i, 'number'])):
        data = data.drop(index=i, axis=0)
data = data.reset_index(drop=True)

In [5]:
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S') # форматирование дат
data['hour'] = data['time'].dt.hour
data['time'] = data['time'].dt.time
data['date'] = pd.to_datetime(data['date'], format='%d.%m.%Y')

In [6]:
df = data.copy()

correct_calls = [] # формируем список, в который будем записывать дни и количество звонков по ним, учитываемые в статистике

correct_data = df['date'].unique()
for i in range(len(list(df['date'].unique()))): # 0:10 - время, которое менеджер должен удерживать клиента на линии
    correct_calls.append(df.loc[(df.loc[:, 'long'] >= "0:10") & (df.loc[:, 'date'] == df['date'].unique()[i])]['number'].count())

d = {'date': correct_data, 'calls': correct_calls}

idx = []
for i in range(len(list(df['date'].unique()))):
    idx.append(i)
    
correct_data = pd.DataFrame(data=d, index=idx)    

In [7]:
display(correct_data)

,date,calls
0,2022-09-01,60
1,2022-09-02,83
2,2022-09-05,26
3,2022-09-06,140
4,2022-09-07,134
5,2022-09-08,73
6,2022-09-09,14
7,2022-09-12,134
8,2022-09-13,77
9,2022-09-14,137


In [8]:
# формируем список дней, не принимаемых в расчет статистики
trip_days = ['2022-09-01', '2022-09-02', '2022-09-05', '2022-09-08', '2022-09-09', '2022-09-13', '2022-09-20', '2022-09-28', '2022-09-30']
display(correct_data.query('date not in @trip_days'))
print('Среднее количество звонков за август', correct_data.query('date not in @trip_days').mean())

,date,calls
3,2022-09-06,140
4,2022-09-07,134
7,2022-09-12,134
9,2022-09-14,137
10,2022-09-15,133
11,2022-09-16,125
12,2022-09-19,132
14,2022-09-21,131
15,2022-09-22,130
16,2022-09-23,123


Среднее количество звонков за август calls    131.461538
dtype: float64


/var/folders/vr/wml9hs7s5wzchnjvgzn6ddjc0000gn/T/ipykernel_7272/1157316885.py:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  print('Среднее количество звонков за август', correct_data.query('date not in @trip_days').mean())
